In [1]:
import sys
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
import importlib
sys.path.append("../")
import metrics
importlib.reload(metrics)

<module 'metrics' from '/Users/narek/Research/library/metrics/metrics/test/../metrics.py'>

In [3]:
%load_ext line_profiler

In [4]:
nC, nI = 1, 1
nL, nLx, nLy, nH = 3, 4, 5, 24

In [5]:
data = np.random.random((100, 10))
Y, X, Z = data[:, :3].T, data[:, 3:7].T, data[:, 7:].T

Ydata = pd.DataFrame(Y.T, index=pd.date_range(start=dt.datetime(2000, 1, 1), periods=100))
Xdata = pd.DataFrame(X.T, index=pd.date_range(start=dt.datetime(2000, 1, 1), periods=100))

In [6]:
metrics.ols_h_njit(Y[0], X)
%timeit metrics.ols_h(Y[0], X)
%timeit metrics.ols_h_njit(Y[0], X)

13.7 µs ± 25.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
5.32 µs ± 16.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
metrics.ols_b_h_njit(Y, X)
%timeit metrics.ols_b_h(Y, X)
%timeit metrics.ols_b_h_njit(Y, X)

108 µs ± 52.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
26.2 µs ± 42.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
metrics.fit_ardl_h_njit(Y[0:1], X, Z, nC, nI, nLy, nLx)
%timeit metrics.fit_ardl_h(Y[0:1], X, Z, nC, nI, nLy, nLx)
%timeit metrics.fit_ardl_h_njit(Y[0:1], X, Z, nC, nI, nLy, nLx)

130 µs ± 57.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
40.6 µs ± 804 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
metrics.fit_var_h_njit(Y, nC, nL)
%timeit metrics.fit_var_h(Y, nC, nL)
%timeit metrics.fit_var_h_njit(Y, nC, nL)

99.3 µs ± 36.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
59 µs ± 476 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
metrics.lpols_njit(Xdata=X, Ydata=Y, nL=nL, nH=nL)
%timeit metrics.lpols(Xdata=X, Ydata=Y, nL=nL, nH=nL)
%timeit metrics.lpols_njit(Xdata=X, Ydata=Y, nL=nL, nH=nL)

84.7 µs ± 380 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
36.7 µs ± 251 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Irfs

In [12]:
Mdl = metrics.VARm(Ydata, nL=nL).irf(ci='bs', nH=nH)
Bc, Bx, B, U, S = Mdl.Est['Bc'], Mdl.Est['Bx'], Mdl.Est['B'], Mdl.Est['U'], Mdl.Est['S']
A0inv = np.linalg.inv(S)
Psi = metrics.get_Psi_from_Bx(Bx, nH)
nL, nH = Mdl.Spec['nL'], Mdl.Irfs.Spec['nH']
Y0 = Y[:, :nL]

In [13]:
%timeit metrics.sim_var(Y0, Bc, Bx, U)
%timeit metrics.sim_var_njit(Y0, Bc, Bx, U)

229 µs ± 776 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
27.1 µs ± 7.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit metrics.get_Psi_from_Bx(Bx, nH)
%timeit metrics.get_Psi_from_Bx_njit(Bx, nH)

139 µs ± 2.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
19.3 µs ± 426 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
%timeit metrics.get_sirf_from_irf(Bx, A0inv)
%timeit metrics.get_sirf_from_irf_njit(Bx, A0inv)

9.53 µs ± 65.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


/Users/narek/Research/library/metrics/metrics/test/../metrics.py:1016: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=True))
  Irf[h] = Psi[h]@Impact


The slowest run took 9.94 times longer than the fastest. This could mean that an intermediate result is being cached.
11.6 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit metrics.get_irfs_VARm(Bx, A0inv, nH)
%timeit metrics.get_irfs_VARm_njit(Bx, A0inv, nH)

27.4 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
24.7 µs ± 235 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Bootstrap vs Wild Bootstrap

In [14]:
%lprun -f metrics.bs_irf Mdl.irf(ci='bs', nH=nH)

Timer unit: 1e-09 s

Total time: 0.015513 s
File: /Users/narek/Research/library/metrics/metrics/test/../metrics.py
Function: bs_irf at line 1026

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1026                                           def bs_irf(Y, U, B, /, *, model_spec, irf_spec):
  1027       100      39000.0    390.0      0.3      nC, nL, nY, nT, dfc = model_spec['nC'], model_spec['nL'], model_spec['nY'], model_spec['nT'], model_spec['dfc']
  1028       100      18000.0    180.0      0.1      nH, ci = irf_spec['nH'], irf_spec['ci']
  1029       100      34000.0    340.0      0.2      Y0 = Y[:, :nL]
  1030       100     789000.0   7890.0      5.1      Bc, Bx = split_C_B(B, nC, nL, nY)
  1031       100      11000.0    110.0      0.1      if ci == 'bs':
  1032       100    1221000.0  12210.0      7.9          idx_r = np.random.choice(nT, size=nT)
  1033       100     559000.0   5590.0      3.6          U_ = U[:, idx_r]
  1034       100      21000.0    210.0     

In [15]:
%lprun -f metrics.bs_irf Mdl.irf(ci='wbs', nH=nH)

Timer unit: 1e-09 s

Total time: 0.016992 s
File: /Users/narek/Research/library/metrics/metrics/test/../metrics.py
Function: bs_irf at line 1026

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1026                                           def bs_irf(Y, U, B, /, *, model_spec, irf_spec):
  1027       100      43000.0    430.0      0.3      nC, nL, nY, nT, dfc = model_spec['nC'], model_spec['nL'], model_spec['nY'], model_spec['nT'], model_spec['dfc']
  1028       100      18000.0    180.0      0.1      nH, ci = irf_spec['nH'], irf_spec['ci']
  1029       100      43000.0    430.0      0.3      Y0 = Y[:, :nL]
  1030       100    1043000.0  10430.0      6.1      Bc, Bx = split_C_B(B, nC, nL, nY)
  1031       100      18000.0    180.0      0.1      if ci == 'bs':
  1032                                                   idx_r = np.random.choice(nT, size=nT)
  1033                                                   U_ = U[:, idx_r]
  1034       100       8000.0     80.0     